# **PRELOAD THE DATAS TO PARC**

Upload csv-file with 2 industry to test parcer.

In [ ]:
! git clone "https://github.com/AlexArutiunian/hhru"

In [ ]:
! pip install fake-useragent icecream

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from icecream import ic
import json
import time

To scrape all types of industryes use "hhru/fields_all.csv" instead of "hhru/fields.csv"

In [ ]:
df_test = pd.read_csv("hhru/fields.csv")
df_test

,id,name_field
0,19,"Телекоммуникации, связь"
1,37,"Общественная деятельность, партии, благотворит..."


# **PARCER**

1 Step.

Parcing links on vacancyes trough info about industry ("id_field" and "prof_role" in code below).

INPUT: 2 files

"hhru/fields.csv"
"hhru/spec.csv"

OUTPUT: json file with structure see below in code

"vacancy_links.json"



In [ ]:
data_fields = pd.read_csv("hhru/fields.csv")
data_spec = pd.read_csv("hhru/spec.csv")

ua = UserAgent()

all_vacancy = []

for index, row in data_fields.iterrows():
    id_field = (int)(row["id"])
    for index2, row2 in data_spec.iterrows():

        industry = row["name_field"]
        prof_role = row2["name_spec"]
        id_spec = row2["id"]

        headers = {
            "User-Agent": ua.random
        }

        print(id_field, id_spec)
        for i in range(0, 20):

            url = (f"https://hh.ru/search/vacancy?area=113&search_field=name&search_field=company_name/"
                   f"&search_field=description&enable_snippets=false&L_save_area=true&/"
                   f"industry={id_field}&professional_role={id_spec}&page={i}")


            req = requests.get(url, headers=headers)

            soup = BeautifulSoup(req.text, "html.parser")

            items_vacancy = soup.find_all("span", class_="serp-item__title-link-wrapper")

            for item in items_vacancy:
                print(item.get_text())
                print(item.find("a").get("href"))
                print(url)
                data_links_vacancy = {
                    "vacancy_name" : item.get_text(),
                    "link": item.find("a").get("href"),
                    "industry": industry,
                    "professional_role": prof_role,
                    "link_list_vacancy_where_it_is": url,
                    "id_pair": (id_field, id_spec)
                }
                # ic(data_links_vacancy)
                if data_links_vacancy not in all_vacancy:
                    all_vacancy.append(data_links_vacancy)

        with open("vacancy_links.json", "w", encoding="utf-8") as fp:
            json.dump(all_vacancy, fp, ensure_ascii=False, indent=4)

In [ ]:
! ls

2 Step.

INPUT: data-file from 1 step

```
path = "vacancy_links.json"
```

OUTPUT: data's resulted for parcer
```
path_out = "test_vac_links.json"
```

In [ ]:
def parc_vac(url):
    print(url)

    ua = UserAgent()
    headers = {
        "User-Agent": ua.random
    }

    try:
        req = requests.get(url, headers=headers)
        req.raise_for_status()

        soup = BeautifulSoup(req.text, "html.parser")

        name_vacancy = soup.find("h1", class_="bloko-header-section-1")
        salary = soup.find("span", class_="bloko-header-section-2 bloko-header-section-2_lite")
        experience = soup.find("p", class_="vacancy-description-list-item")
        name_company_tag = soup.find_all("span", class_="bloko-header-section-2 bloko-header-section-2_lite")
        name_company = name_company_tag[1].get_text().replace("\xa0", " ") if len(name_company_tag) > 1 else ""
        desc = soup.find("div", class_="vacancy-description").get_text().replace("О компании", "") if soup.find("div", class_="vacancy-description") else ""
        time_creation = soup.find("p", class_="vacancy-creation-time-redesigned").get_text().replace("\xa0", " ") if soup.find("p", class_="vacancy-creation-time-redesigned") else ""

        skills_divs = soup.find_all("div", class_="bloko-tag bloko-tag_inline")
        skills = [item.get_text().replace("\xa0", " ") for item in skills_divs]

        data = {
            "name_vacancy": name_vacancy.get_text() if name_vacancy else "",
            "salary": salary.get_text().replace("\xa0", " ") if salary else "",
            "experience": experience.get_text() if experience else "",
            "name_company": name_company,
            "description": desc,
            "time_creation": time_creation,
            "skills": skills
        }

        ic(data)

        return data

    except Exception as e:
        with open("err.txt", "a", encoding="utf-8") as fp:
            fp.write(f"{e} field {data.get('id_pair', [])[0]} spec {data.get('id_pair', [])[1]} vac name {data.get('link', '')} vac name {data.get('vacancy_name', '')}\n")
        return None

path = "vacancy_links.json"
path_out = "test_vac_links.json"

with open(path, "r", encoding="utf-8") as fp:
    datas = json.load(fp)

datas_out = []

for i, data in enumerate(datas):
    print(f"Processing: {i}/{len(datas)}")

    try:
        result = {**data, **parc_vac(data["link"])}
        if result:
            datas_out.append(result)
    except Exception as e:
        print(f"Error processing item: {e}")

if datas_out:
    with open(path_out, "w", encoding="utf-8") as fp:
        json.dump(datas_out, fp, ensure_ascii=False, indent=2)

Processing: 0/1074
https://adsrv.hh.ru/click?b=877824&place=35&meta=bf4RII5waWOgB879mVptsiJ4C1d9BGu44zd7DiLt-1eQua96SJBdDwp3I8nubmRLSi0RO7yxYDMggWnwhK0LBwpI0xixbAXn5PusI-FrjPLGI0V0b3cogGjAzEgJTrRTzVj_Fk7CTesh5YiLhW7QTL5jTdx7hMDWTvb83SUg3-QNYtOYFiuDL5qSNf1Z5B54WXRB0cHAWTryS84OmtwGHqwSAi0WssBjGiDdT8_VgjSIEJdYglZACs1tSn9cRReRB7Ic0dR4yEf_vP3N3sAkBc-Eg70q1D2-X_DLoJ66GosV4lFzFEF5TMu-JmzeebTfd4IGUH7KiwlTFDJEVJ0l_fc6gXtm2uY4gYXMfV_bDmJ3yR5FkZUv3yb6kUuZPTDs2QKl6VfBNzOyNxpDo1iy2779mVZNgsfnfRB3oGDcru9UvW0kjWeKKdFEvLqcjMQSAMApk1d1Byatkv_y8bl_LZcIdluOWr5q0yI8Kw7z_JvzZJU_xMpzF_d9t4EyFGGY4gpApubq8Gmbn-_nW50U5g%3D%3D&clickType=link_to_vacancy


ic| data: {'description': 'АО "ТОЧИНВЕСТ" приглашает в свою команду Менеджера! Мы '
                          'предлагаем Вам:  Преимущества работы в крупной стабильной '
                          'компании, возможность карьерного и личностного роста. '
                          'Официальное трудоустройство согласно ТК РФ, все социальные '
                          'гарантии. Программа лояльности. Достойная, своевременная '
                          'заработная плата. Премии за непрерывный стаж работы на '
                          'предприятии. Материальная поддержка сотрудников в различных '
                          'жизненных ситуациях. График работы 5/2, с 8.30 до 17.00. Мы '
                          'работаем в современном офисе, где приятно решать '
                          'бизнес-задачи и взаимодействовать с коллегами; '
                          'Наставничество, обучение и развитие вместе с компанией; '
                          'Дополнительное медицинское страхование; Бону

Processing: 1/1074
https://adsrv.hh.ru/click?b=871213&place=36&meta=WgeWsIHE2hc7kymI7Td-Iq2ZETueGMaPZhxPj4ohYLDr6DezsoKhu5mwLcW7fqd7DEEMqvmexysAnfXpVoKRdghQEvvSAfLhavFLDxfneHVoh22Dp7M4ibgti4VSzQ6kFcQFcfVSC0OwvYm7FKIqbZMz6ILBJagG7nSPH2EFkithmjD9Sryiyjw7BUaOWa7Ukh9sVE0INj_i6DX2uCmOLfgWfio6abbXN22waYPacS4XrKuHDzgB2DAZ_P5S0rB_A4e2I6mCijoQ4xiqJ473QQm9UV6o-kaBJ1NzSJOk4y5nHPPUgY-BudJsbUHI06KVgF1rAEGADgAYPi5EixJXHN-K1cck5G0wLVrDjnrSUS_NngyeaQEfSyDM1pQwIVeQE6g_4Jsbzf1OCnN-ofF004P6eEvJBWXCCWEto0QXbBPxu9Oc6FlRwS5rEdNsllEHeYN0rQ1-WcSdQfAVH3-28E-o0WPugS0u4I14Tn-P63sFpmn5nFm5mFyh1YZt6U1ThUkYI0XfACw9p5pAAhIo2Q%3D%3D&clickType=link_to_vacancy


ic| data: {'description': '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '
                          '
          '


KeyboardInterrupt: 